<a href="https://colab.research.google.com/github/hemanthmeruga/IEApp/blob/master/R6_InternalLab_AIML_HM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
from sklearn.preprocessing import Normalizer

### Collect Data

In [0]:
import pandas as pd

In [0]:
def mount_drive(filename):
  from google.colab import drive
  drive.mount('/gdrive/')
  data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/' + filename)
  return data

In [0]:
data =mount_drive("prices.csv")

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).


### Check all columns in the dataset

In [0]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [0]:
data.shape

(851264, 7)

In [0]:
data.describe()

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,70.836986,70.857109,70.118414,71.543476,5.415113e+06
std,83.695876,83.689686,82.877294,84.465504,1.249468e+07
min,0.850000,0.860000,0.830000,0.880000,0.000000e+00
25%,33.840000,33.849998,33.480000,34.189999,1.221500e+06
50%,52.770000,52.799999,52.230000,53.310001,2.476250e+06
75%,79.879997,79.889999,79.110001,80.610001,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08


### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],axis=1,inplace=True)

### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data = data.head(1000)

In [0]:
data.shape

(1000, 5)

In [0]:
data.sample(5)

,open,close,low,high,volume
68,115.809998,117.989998,115.099998,118.010002,621700.0
597,40.259998,40.349998,40.259998,40.630001,546500.0
232,120.690002,121.480003,120.180000,122.059998,604700.0
415,82.610001,83.459999,82.550003,83.599998,18082400.0
607,26.139999,25.799999,25.719999,26.280001,2792600.0


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

X = data.drop(columns= "close" , axis = 1) 
Y = data["close"]
transformer = Normalizer()
X= transformer.fit_transform(X)

train_x, test_x,train_y,test_y = train_test_split(X, Y, test_size=.3, random_state=7)




## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
X1 = tf.placeholder(dtype=tf.float32, shape=[None, 4])
Y1 = tf.placeholder(dtype=tf.float32, shape=[None])

2.Define Weights and Bias

In [0]:
w = tf.Variable(tf.zeros(shape=[4,1],dtype="float32"))
b = tf.Variable(tf.zeros(shape=(1),dtype="float32"))

3.Prediction

In [0]:
out = tf.add(tf.matmul(X1, w), b)

4.Loss (Cost) Function [Mean square error]

In [0]:
mse = tf.reduce_mean(tf.squared_difference(out, Y1))

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
opt=tf.train.GradientDescentOptimizer(0.01).minimize(mse)


## Execute the Graph for 100 epochs and observe the loss

In [0]:
net = tf.Session()
net.run(tf.global_variables_initializer())

In [0]:

for i in range(100):
    _,loss = net.run([opt,mse], feed_dict={X1: train_x, Y1: train_y})
    print('Current Loss on iteration',i,loss)

Current Loss on iteration 0 8370.582
Current Loss on iteration 1 8002.651
Current Loss on iteration 2 7663.5664
Current Loss on iteration 3 7351.0664
Current Loss on iteration 4 7063.0654
Current Loss on iteration 5 6797.644
Current Loss on iteration 6 6553.032
Current Loss on iteration 7 6327.598
Current Loss on iteration 8 6119.837
Current Loss on iteration 9 5928.3643
Current Loss on iteration 10 5751.9043
Current Loss on iteration 11 5589.278
Current Loss on iteration 12 5439.4014
Current Loss on iteration 13 5301.276
Current Loss on iteration 14 5173.9785
Current Loss on iteration 15 5056.662
Current Loss on iteration 16 4948.5425
Current Loss on iteration 17 4848.9004
Current Loss on iteration 18 4757.0693
Current Loss on iteration 19 4672.438
Current Loss on iteration 20 4594.442
Current Loss on iteration 21 4522.5615
Current Loss on iteration 22 4456.3154
Current Loss on iteration 23 4395.263
Current Loss on iteration 24 4338.997
Current Loss on iteration 25 4287.143
Current Lo

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [0]:
net.run(w).shape

(4, 1)

In [0]:
net.run(w)

array([[2.5353241e-03],
       [2.5108382e-03],
       [2.5564777e-03],
       [3.3674850e+01]], dtype=float32)

In [0]:
net.run(b).shape

(1,)

In [0]:
net.run(b)

array([33.674854], dtype=float32)

In [0]:
net.close()

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
from keras.models import Sequential
from keras.layers import Dense

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1,  input_shape=(4,)))


### Execute the model

In [0]:
model.compile(optimizer='sgd', loss='mean_squared_error')


In [0]:
from sklearn.model_selection import train_test_split

X = data.drop(columns= "close" , axis = 1) 
Y = data["close"]
transformer = Normalizer()
X= transformer.fit_transform(X)

train_x, test_x,train_y,test_y = train_test_split(X, Y, test_size=.3, random_state=7)


In [0]:
model.fit(train_x,train_y, epochs=100)

ValueError: ignored

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1_2 (Ba multiple                  16        
_________________________________________________________________
dense_4 (Dense)              multiple                  5         
Total params: 21
Trainable params: 13
Non-trainable params: 8
_________________________________________________________________


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris= mount_drive("Iris-1.csv")

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).


In [0]:
iris.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

In [0]:
iris.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


In [0]:
iris["Species"] = pd.Categorical(iris["Species"]).codes

In [0]:
iris.drop("Id",axis=1,inplace=True)

### Splitting the data into feature set and target set

In [0]:
features = iris.drop(columns= ['Species']) 

target = iris[['Species']]


In [0]:
features.head(2)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


In [0]:
target.head(2)

,Species
0,0
1,0


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

### Divide the dataset into Training and test (70:30)

In [0]:
from sklearn.model_selection import train_test_split
transformer = Normalizer()
#features = transformer.fit_transform(features)

train_x, test_x,train_y,test_y = train_test_split(features, target, test_size=.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [0]:

#train model
model.fit(features.values, target.values, validation_split=0.2, epochs=30)

Train on 120 samples, validate on 30 samples
Epoch 1/30
120/120 [==============================] - 0s 1ms/sample - loss: 2.4904 - acc: 0.0000e+00 - val_loss: 7.9632 - val_acc: 0.0000e+00
Epoch 2/30
120/120 [==============================] - 0s 142us/sample - loss: 2.4862 - acc: 0.0333 - val_loss: 7.6380 - val_acc: 0.0000e+00
Epoch 3/30
120/120 [==============================] - 0s 146us/sample - loss: 2.4831 - acc: 0.0500 - val_loss: 7.4335 - val_acc: 0.0000e+00
Epoch 4/30
120/120 [==============================] - 0s 159us/sample - loss: 2.4814 - acc: 0.0583 - val_loss: 7.2702 - val_acc: 0.0000e+00
Epoch 5/30
120/120 [==============================] - 0s 153us/sample - loss: 2.4802 - acc: 0.0917 - val_loss: 7.1517 - val_acc: 0.0000e+00
Epoch 6/30
120/120 [==============================] - 0s 146us/sample - loss: 2.4781 - acc: 0.1167 - val_loss: 7.0617 - val_acc: 0.0000e+00
Epoch 7/30
120/120 [==============================] - 0s 143us/sample - loss: 2.4780 - acc: 0.1417 - val_loss: 6.

### Report Accuracy of the predicted values

In [0]:
model.predict(test_x)

array([[0.31266114, 0.3359437 , 0.35139513],
       [0.30647665, 0.32860282, 0.3649205 ],
       [0.2697001 , 0.31608474, 0.4142151 ],
       [0.30983818, 0.33588454, 0.35427728],
       [0.33724383, 0.3301039 , 0.3326523 ],
       [0.29762256, 0.3450849 , 0.35729256],
       [0.302427  , 0.3947364 , 0.3028366 ],
       [0.3221134 , 0.35264298, 0.32524362],
       [0.27069396, 0.32167685, 0.40762922],
       [0.30681744, 0.36379626, 0.32938626],
       [0.3108494 , 0.37864232, 0.31050825],
       [0.29403183, 0.33183616, 0.374132  ],
       [0.234098  , 0.32249752, 0.44340444],
       [0.28217614, 0.35773295, 0.3600909 ],
       [0.26453003, 0.32299492, 0.41247505],
       [0.29544476, 0.3307112 , 0.3738441 ],
       [0.33089504, 0.36824295, 0.30086195],
       [0.31355184, 0.36431384, 0.32213432],
       [0.24862848, 0.3213934 , 0.42997816],
       [0.2801069 , 0.3274986 , 0.39239445],
       [0.32221147, 0.35943076, 0.31835774],
       [0.30521223, 0.36602014, 0.32876766],
       [0.